# Group Project, Attempting to get RNN LSTM to work.

In [1]:
import json, os, re, shutil, sys, time
import collections, itertools
import unittest
from IPython.display import display, HTML

# NLTK for NLP utils and corpora
import nltk

# NumPy and TensorFlow
import numpy as np
import tensorflow as tf
assert(tf.__version__.startswith("1."))
import string          # various string functions
import difflib         # classes and functions for comparing sequences
import utils           # word processing functions and distance functions, pretty printing and data loading

from importlib import reload       # reload external files

# utils.pretty_print_matrix uses Pandas. Configure float format here.
import pandas as pd
pd.set_option('float_format', lambda f: "{0:.04f}".format(f))

# rnn code
import rnnlstm

# sklearn
import sklearn
from sklearn.model_selection import train_test_split

# Helper libraries
from shared_lib import vocabulary, tf_embed_viz

# set a default vocab size
V=20000

import plotly.plotly
import plotly.graph_objs as go

# need to look into plotly issues with this line in python3
# plotly.offline.init_notebook_mode() # run at the start of every ipython notebook

from sklearn.metrics import log_loss    # used in measurement / scoring
from sklearn.metrics import classification_report   # among other things, provides accuracy and f1

# Your code
# import rnnlm
# import rnnlm_test
# reload(rnnlm)
# reload(rnnlm_test)

In [2]:
# Generate training data dataframe
# train_lite for testing approach
train = pd.read_csv('Data/train_lite.csv') #index_col=0
# full train for when working
# train = pd.read_csv('Data/train.csv') #index_col=0

print (len(train))


10000


In [3]:
del train['Unnamed: 0']
del train['Unnamed: 0.1']
train.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,57674,5725,38477,How can I get a list of my Gmail accounts?,How do I find my list of GMail addresses?,1
1,185568,283254,283255,What are the symptoms of child abuse?,What are signs of child abuse?,1
2,359111,400936,488715,What are some examples of selfishness?,Do we all occasionally tend to emotional masoc...,0
3,204216,281934,307050,Is Qnet a scam?,Where is the registered office of Qnet in Mumbai?,0
4,5649,11103,11104,What do Americans think about Donald Trump?,What do you think about Donald Trump pick?,1


In [4]:
reload(vocabulary)
reload(utils)
question1 = train.question1.tolist()
question2 = train.question2.tolist()
is_duplicate = train.is_duplicate.tolist()

# print('Question pairs: %d' % len(question1))

# build word index
questions = question1 + question2

vocab = utils.build_vocab(questions, V)

In [5]:
# print(vocab.ordered_words())

In [18]:
word_index = vocab.ordered_words()

print("Words in index: %d" % len(word_index))

Words in index: 20000


In [75]:
vocab.word_to_id.get('<unk>')

2

In [6]:
# critical definition
# maps all words to ID

wordset =  vocab.word_to_id
print (len(wordset))
print (type(wordset))
print (wordset.get('paying'))

20000
<class 'dict'>
None


In [14]:
def process_questions(questions, wordset, vocab):
    sents = ["<s>" + s + "</s>" for s in questions]
    # print (sents[0:5])
 
    sequence = []
    num_sequence = []
    for s in sents:
        # for each sentence in list of sentences
        # print (s)
        s_words = []
        s_w_ids = []
        for w in s.split():
            # capture list of words for sentence
            # once all captured, add to list with sequence
            
            # for each word in split sentence
            # print (w)
            w = w.lower()
            if (wordset == None) or (w in wordset): 
                s_words.append(w)
                s_w_ids.append(vocab.word_to_id.get(w))
            else:
                s_words.append("<unk>") # unknown token
                s_w_ids.append(vocab.word_to_id.get("<unk>"))
                               
            '''
            if w.isdigit():
            if (wordset != None) and (w in wordset):
                    q1_sequence.append(w)

                w = utils.canonicalize_digits(w) # try to canonicalize numbers
            '''

        sequence.append(s_words)
        num_sequence.append(s_w_ids)
        
        
    sequence = np.array(sequence)
    num_sequence = np.array(num_sequence)
    return sequence, num_sequence

In [15]:
q1_sequences, q1_seq_num = process_questions(question1, wordset, vocab)
q2_sequences, q2_seq_num = process_questions(question2, wordset, vocab)

In [16]:
q1_sequences

array([ ['<unk>', 'can', '<unk>', 'get', 'a', 'list', '<unk>', 'my', '<unk>', '<unk>'],
       ['<unk>', 'are', 'the', 'symptoms', '<unk>', 'child', '<unk>'],
       ['<unk>', 'are', 'some', 'examples', '<unk>', '<unk>'], ...,
       ['<unk>', 'does', 'gases', 'heat', 'up', 'if', 'we', 'try', 'to', 'cool', 'it', '<unk>', 'expanding', 'it', 'above', 'inversion', '<unk>'],
       ['<unk>', 'missed', 'the', 'call', 'on', 'brexit;', 'how', 'accurate', 'are', 'current', 'polling', 'methods', 'and', 'could', 'we', 'see', '<unk>', 'forecasting', 'flubs', 'with', 'the', 'presidential', '<unk>'],
       ['<unk>', 'can', 'be', '<unk>', 'about', 'the', 'hyperinflation', '<unk>', 'zimbabwe?', 'what', 'were', 'the', 'causes', '<unk>', 'it', 'and', 'its', 'impact', 'on', 'the', 'economy', '<unk>', 'the', '<unk>']], dtype=object)

In [17]:
q2_seq_num

array([[2, 2, 2, 4535, 1088, 18597, 2, 2, 2],
       [2, 10517, 4520, 2, 15722, 2],
       [2, 11222, 12316, 11923, 13401, 18943, 5987, 2], ...,
       [2, 4136, 17114, 653, 13653, 2206, 2, 2, 14510, 15509, 16553, 2],
       [2, 6364, 12210, 8754, 13427, 14284, 9808, 18943, 18191, 7580, 1603, 8086, 1410, 3557, 10621, 14510, 7580, 3402, 19665, 15509, 16553, 408, 12772, 4666, 10096, 19465, 13494, 905, 1410, 5033, 3665, 2],
       [2, 2, 1323, 4585, 2, 7580, 12838, 12097, 19665, 2]], dtype=object)

In [18]:
print (len(q1_sequences))
print (len(q2_sequences))

# NOTE, with sequences, each original sentence is a list within the list

10000
10000


In [19]:
reload (rnnlstm)

# building graph based on a3 assigment
TF_GRAPHDIR = "tf_graph"

# Clear old log directory.
shutil.rmtree(TF_GRAPHDIR, ignore_errors=True)

# V, H, and num_layers are all things we can play with...
# default corpus is 'brown'
# for now, want to see running with this data
# set hyperparameters
lm = rnnlstm.RNNLM(V=V, H=200, num_layers=2)

lm.BuildCoreGraph()
lm.BuildTrainGraph()
lm.BuildSamplerGraph()
summary_writer = tf.summary.FileWriter(TF_GRAPHDIR, lm.graph)

In [20]:
#X = np.stack((q1_sequences, q2_sequences), axis=1)
X = np.stack((q1_seq_num, q2_seq_num), axis=1)

#y = is_duplicate
y = np.array(is_duplicate)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
Q1_train = X_train[:,0]
Q2_train = X_train[:,1]
Q1_test = X_test[:,0]
Q2_test = X_test[:,1]

In [21]:
Q1_train[5]

[2, 10517, 13653, 2, 13494, 1005, 15553, 4010, 2]

In [59]:
# max_time = 100

# not sure these matter any more:
max_time = 50
# sentences with > 50 words will be truncated to 50 words
# rest are padded to 50

# for our purposes, a batch is a question pair
# this varies wether train or test though...
# something to think about
batch_size = len(Q1_train)
print (batch_size)


9000


In [60]:
def pad_sent(q, max_time):
    if len(q) > max_time:
        # cut it off
        q = q[0:max_time]
    else:
        # pad to max_time
        amt_to_pad = max_time - len(q)
        q.extend([2] * amt_to_pad)
    return q
    

In [64]:
def batch_generator(q1, q2, label, max_time):
    for i in range(len(q1)):
        # # of q1 and q2 questions is the same, so can use one to iterate
        
        # padded or trunctating all sentences to max_time
        q1[i] = pad_sent(q1[i], max_time)
        q2[i] = pad_sent(q2[i], max_time)
        
    # now we will make them 1 list (vs 2)
    q1 = np.array([idx for sublist in q1 for idx in sublist] )
    q2 = np.array([idx for sublist in q2 for idx in sublist] )
    
    # print (type(q1))
    # print (q1.shape)
        
    q1 = q1.reshape([batch_size,-1])
    q2 = q2.reshape([batch_size,-1])
    label = label.reshape([batch_size, -1])
    
    # print (q1.shape)
    # print (q1.shape[1])
    
    # need to look at label, why is it 9000???
    
    #print (label.shape)
    # Yield batches
    # yield q1[i], q2[i], label[i]
    # return q1, q2, label

    # Yield batches
    for i in range(0, q1.shape[1], max_time):
        yield q1[:,i:i+max_time], q2[:,i:i+max_time], label[:,i:i+max_time]

In [65]:
# possible add another for loop, that allows you to move a sentence over multiple
# need to look at what batch_generator was providing in a3 (perhaps on the cloud even)

In [66]:
q1 = [2, 4660, 2, 555, 7902, 2, 2, 1804, 1908, 10320, 283, 2, 4842, 2]
#q1.reshape([batch_size,-1])
for i in range(0, len(q1), max_time):
    print (i)
    print (type(q1))
    print (type(max_time))
    #print(q1[:,i:i+max_time])

0
<class 'list'>
<class 'int'>


In [67]:
# testing batch generator
bi = batch_generator(Q1_train, Q2_train, y_train, max_time)
for i, (w1, w2, y) in enumerate(bi):
    print ("i:", i)
    print ("w1:", w1)
    print ("w2:", w2)
    print ("y:", y)

i: 0
w1: [[    2 10517 13653 ...,     2     2     2]
 [    2 13605  7580 ...,     2     2     2]
 [    2 13605 13494 ...,     2     2     2]
 ..., 
 [    2 13605 13494 ...,     2     2     2]
 [    2 19043     2 ...,     2     2     2]
 [    2 13605 19763 ...,     2     2     2]]
w2: [[    2 13605 13494 ...,     2     2     2]
 [    2 13494  1005 ...,     2     2     2]
 [    2 13605 13494 ...,     2     2     2]
 ..., 
 [    2   164 13494 ...,     2     2     2]
 [    2 19043 12772 ...,     2     2     2]
 [    2 13605 13494 ...,     2     2     2]]
y: [[1]
 [0]
 [1]
 ..., 
 [1]
 [0]
 [0]]


In [68]:
def run_epoch(lm, session, batch_iterator,
              train=False, verbose=False,
              tick_s=10, learning_rate=0.1):
    start_time = time.time()
    tick_time = start_time  # for showing status
    total_cost = 0.0  # total cost, summed over all words
    total_batches = 0
    total_words = 0

    if train:
        train_op = lm.train_step_
        use_dropout = True
        loss = lm.train_loss_
    else:
        train_op = tf.no_op()
        use_dropout = False  # no dropout at test time
        loss = lm.loss_  # true loss, if train_loss is an approximation
        
    # How do I bring in both w's? Getting a "need to pass W_1 through cost, _, h" function. No idea.

    for i, (w1, w2, y) in enumerate(batch_iterator):
        cost = 0.0
        # At first batch in epoch, get a clean initial state.

        if i == 0:
            h = session.run(lm.initial_h_, {lm.input_w_q1_: w1, lm.input_w_q2_: w2})

       
        feed_dict = {lm.input_w_q1_: w1, lm.input_w_q2_: w2, lm.target_y_: y, lm.initial_h_: h}
        cost, _, h = session.run([lm.train_loss_, lm.train_step_, lm.final_h_], feed_dict=feed_dict)

        total_cost += cost
        total_batches = i + 1
        total_words += batch_size * max_time

        if verbose and (time.time() - tick_time >= tick_s):
            avg_cost = total_cost / total_batches
            avg_wps = total_words / (time.time() - start_time)
            print ("[batch %d]: seen %d words at %d wps, loss = %.3f" %
                i, total_words, avg_wps, avg_cost)
            tick_time = time.time()  # reset time ticker

    return total_cost / total_batches

In [69]:
# ids are the questions, but are we passing both at once?
# from what i can gather, this was built for one id at a time. will need to pass through both questions at once.
reload(utils)
def score_dataset(lm, session, Q1, Q2, y, name="Data"):
    # For scoring, we can use larger batches to speed things up.
    # bi = utils.batch_generator(Q1, Q2, y, batch_size=50, max_time=100)
    bi = batch_generator(Q1, Q2, y, max_time)
    cost = run_epoch(lm, session, bi, 
                     learning_rate=1.0, train=False, 
                     verbose=False, tick_s=3600)
    print ("%s: avg. loss: %.03f  (perplexity: %.02f)" % (name, cost, np.exp(cost)))

In [70]:
# Training parameters


#######

learning_rate = 0.5
num_epochs = 10

# Model parameters
model_params = dict(V=vocab.size, 
                    H=100, 
                    softmax_ns=200,
                    num_layers=1)

TF_SAVEDIR = "tf_saved"
checkpoint_filename = os.path.join(TF_SAVEDIR, "rnnlstm")
trained_filename = os.path.join(TF_SAVEDIR, "rnnlstm_trained")

In [71]:
reload (rnnlstm)
reload (utils)

# Will print status every this many seconds
print_interval = 5

# Clear old log directory
shutil.rmtree("tf_summaries", ignore_errors=True)

lm = rnnlstm.RNNLM(**model_params)
lm.BuildCoreGraph()
lm.BuildTrainGraph()

# Explicitly add global initializer and variable saver to LM graph
with lm.graph.as_default():
    initializer = tf.global_variables_initializer()
    saver = tf.train.Saver()
    
# Clear old log directory
shutil.rmtree(TF_SAVEDIR, ignore_errors=True)
if not os.path.isdir(TF_SAVEDIR):
    os.makedirs(TF_SAVEDIR)

with tf.Session(graph=lm.graph) as session:
    # Seed RNG for repeatability
    tf.set_random_seed(42)

    session.run(initializer)

    for epoch in range(1,num_epochs+1):
        t0_epoch = time.time()
        #bi = utils.batch_generator(Q1_train, Q2_train, y_train, batch_size, max_time)
        bi = batch_generator(Q1_train, Q2_train, y_train, max_time)
        print ("[epoch %d] Starting epoch %d" % (epoch, epoch))

        # Run a training epoch.
        
        run_epoch(lm, session, bi, train=True) 

        print ("[epoch %d] Completed in %s" % (epoch, utils.pretty_timedelta(since=t0_epoch)))
    
        # Save a checkpoint
        saver.save(session, checkpoint_filename, global_step=epoch)
    
        ##
        # score_dataset will run a forward pass over the entire dataset
        # and report perplexity scores. This can be slow (around 1/2 to 
        # 1/4 as long as a full epoch), so you may want to comment it out
        # to speed up training on a slow machine. Be sure to run it at the 
        # end to evaluate your score.
        print ("[epoch %d]" % epoch),
        #score_dataset(lm, session, train_ids, name="Train set")
        print ("[epoch %d]" % epoch),
        score_dataset(lm, session, Q1_test, Q2_test, y_test, name="Test set")
        print ("")
    
    # Save final model
    saver.save(session, trained_filename)

[epoch 1] Starting epoch 1


InvalidArgumentError: assertion failed: [Expected shape for Tensor sequence_length:0 is ] [9000] [ but saw shape: ] [50]
	 [[Node: rnn/Assert/Assert = Assert[T=[DT_STRING, DT_INT32, DT_STRING, DT_INT32], summarize=3, _device="/job:localhost/replica:0/task:0/cpu:0"](rnn/All, rnn/Assert/Assert/data_0, rnn/stack, rnn/Assert/Assert/data_2, rnn/Shape_1)]]

Caused by op 'rnn/Assert/Assert', defined at:
  File "C:\Users\andre\Anaconda2\envs\Python3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\andre\Anaconda2\envs\Python3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\andre\Anaconda2\envs\Python3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\andre\Anaconda2\envs\Python3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\andre\Anaconda2\envs\Python3\lib\site-packages\ipykernel\kernelapp.py"

In [ ]:
# something is going on with pre-defining what is in the tensor, rather than passing as part of the batch i think...
# need to keep looking